# Understanding NDVI and NDSI

Welcome to the first project in the Remote Sensing and Earth Observation chapter of the course. In this project, we will learn about two important indices used in remote sensing: NDVI and NDSI.

## Introduction

Remote sensing is the science of obtaining information about objects or areas from a distance, typically from aircraft or satellites. The data collected by remote sensing instruments can be used to monitor and analyze changes in the environment, such as land use, vegetation cover, and climate patterns.

One of the key applications of remote sensing is the measurement of vegetation health and snow cover. Two commonly used indices for this purpose are the Normalized Difference Vegetation Index (NDVI) and the Normalized Difference Snow Index (NDSI). Other indices like NDWI (Normalized Difference Water Index) are also used for similar purposes.

These indices are calculated using the reflectance values of different bands of the electromagnetic spectrum. In this project, we will learn how to calculate NDVI and NDSI using satellite imagery data and visualize the results.

## Learning Objectives

In this project, you will learn:

- What NDVI and NDSI are and how they are calculated.
- How to calculate NDVI and NDSI using satellite imagery data.
- How to obtain satellite imagery data using Google Earth Engine.
- How to visualize the results of NDVI and NDSI calculations.

To meet these learning objectives, we will be looking at a region of interest (ROI) in the Himalayan region and calculate the NDVI and NDSI values for this area using Landsat 8 satellite imagery data.

### Himalayan Food Security

The Himalayan region is home to a diverse range of ecosystems and is a critical area for food security in South Asia. The region is known for its rich biodiversity, unique landscapes, and important water resources. However, the region is also vulnerable to climate change, with changes in temperature and precipitation patterns affecting agriculture, water availability, and livelihoods.

Monitoring vegetation health and snow cover in the Himalayan region is important for understanding the impact of climate change on food security and water resources. NDVI and NDSI are valuable tools for assessing these changes and providing insights into the health of ecosystems in the region.

## Let's Get Started!

The required libraries will be listed in the next cell for you to install if you don't have them already.

In [8]:
!pip install -q -U earthengine-api geemap

## Google Earth Engine

Google Earth Engine is a cloud-based platform for planetary-scale environmental data analysis. It provides a wide range of datasets, tools, and APIs for analyzing and visualizing geospatial data. In this project, we will use Google Earth Engine to access Landsat 8 satellite imagery data and calculate NDVI and NDSI values for a region of interest in the Himalayan region.

To use Google Earth Engine, you will need to create an account and set up the necessary credentials. You can sign up for a free account at [Google Earth Engine](https://earthengine.google.com/).

Google Earth Engine can produce petabytes of data, so we will use a small region of interest (ROI) in the Himalayan region to demonstrate the calculation of NDVI and NDSI values. The ROI will be defined by a bounding box with specific coordinates for you.

Let's start by importing Google Earth Engine and authenticating.

In [9]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

True

In [10]:
# Initialize the Earth Engine module.
ee.Initialize()

## Initialize Region of Interest and Time Period

Now that we have Google Earth Engine set up, let's define some initial parameters for our analysis. We will define the region of interest (ROI) using a bounding box with specific coordinates for the Himalayan region and define the time period for which we want to analyze the satellite imagery data—Landsat 8 data for winter 2021 and 2022.

We have a choice of tools here to visualize the data. We will use `geemap` in this project as it is full of Google Earth Engine functionalities and is easy to use.

In [11]:
import geemap

# Define a small bounding box for the Pokhara Valley in the Himalayan region
roi = ee.Geometry.Rectangle([83.87, 28.3, 84.02, 28.4])

# Define time periods for winter for different years
winter_2021 = ee.DateRange("2021-12-01", "2022-02-28")
winter_2022 = ee.DateRange("2022-12-01", "2023-02-28")

## Retrieve and Scale Landsat 8 Data

In the next cell, we will retrieve Landsat 8 data for the defined region of interest and time period. We will then scale the data to the appropriate reflectance values before calculating the NDVI and NDSI values and visualizing the RGB image.

We will use Landsat 8 collection 2, tier 1, level 2 data for this analysis filtered by the region of interest, time period, and cloud cover percentage less than 10%.

### Note

Cloud cover is an important factor to consider when analyzing optical satellite imagery data. There are techniques to mask out clouds and cloud shadows from the images to ensure accurate calculations of vegetation indices like NDVI and NDSI, but for this project, we will filter the data based on cloud cover percentage.

Additionally, other remote sensors employ LIDAR and RADAR to "see" through clouds. LIDAR uses light to measure distances so it is effective in conditions where clouds are not so dense. RADAR uses radio waves to detect objects and is also effective in cloudy conditions.

### Instructions

In the next cell:
- Retrieve Landsat 8 data for the defined region of interest, time period, and cloud cover percentage less than 10%.
- Scale the data to the appropriate reflectance values ([see the developer guide](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands)).

In [12]:
# Function to get composite image for a given date range
def get_median_image(date_range: ee.daterange.DateRange) -> ee.Image:
    # Load the Landsat 8 image collection
    image_collection = (
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        # TODO: Filter by the region of interest
        # TODO: Filter by the date range
        # TODO: Filter by cloud cover
    )

    # Return the median image
    return image_collection.median().clip(roi)


# Get composite images for each time period
winter_2021_image = get_median_image(winter_2021)
winter_2022_image = get_median_image(winter_2022)


# Scale the bands (see https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands)
def apply_scale_factors(image: ee.Image) -> ee.Image:
    optical_bands = image.select("SR_B.")  # TODO: Scale factor for optical bands
    thermal_bands = image.select("ST_B.*")  # TODO: Scale factor for thermal bands

    # Return the scaled image with the original bands
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)


# # Apply the scale factors to the images
winter_2021_image = apply_scale_factors(winter_2021_image)
winter_2022_image = apply_scale_factors(winter_2022_image)

## Calculate NDVI and NDSI

Now that we have the Landsat 8 data for the defined region of interest and time period, let's calculate the NDVI and NDSI values using the reflectance values of the appropriate bands.

The NDVI and NDSI formulas are as follows:

- NDVI = (NIR - RED) / (NIR + RED)
- NDSI = (GREEN - SWIR) / (GREEN + SWIR)

Where:
- NIR = Near-Infrared band
- RED = Red band
- GREEN = Green band
- SWIR = Shortwave Infrared band

The required bands for NDVI and NDSI calculations can be found using [the developer guide](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands).

### Instructions

In the next cell:
- Fill out the formulas to calculate NDVI and NDSI using the reflectance values of the appropriate bands.

In [13]:
# Function to calculate NDVI
def calculate_ndvi(image: ee.Image) -> ee.Image:
    ndvi = image  # TODO: Calculate NDVI
    return image.addBands(ndvi.rename("NDVI"))


# Function to calculate NDSI
def calculate_ndsi(image: ee.Image) -> ee.Image:
    ndsi = image  # TODO: Calculate NDSI
    return image.addBands(ndsi.rename("NDSI"))


# Apply the NDVI and NDSI calculations to the images
winter_2021_image = calculate_ndsi(calculate_ndvi(winter_2021_image))
winter_2022_image = calculate_ndsi(calculate_ndvi(winter_2022_image))

## Visualize NDVI and NDSI

Now all that's left is to visualize the NDVI and NDSI values for the defined region of interest and time period. 

Our map will have the following layers:
- RGB images of the Landsat 8 for each year.
- NDVI values for each year.
- NDSI values for each year.

In [14]:
# Create a map using geemap
Map = geemap.Map()
Map.centerObject(roi, 12)

# Add the Landsat layer to the map
vis_params = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],  # Red, Green, Blue bands
    "min": 0.0,
    "max": 0.3,
}

# Visualize the NDVI and NDSI bands
ndvi_ndsi_vis_params = {
    "min": -1,
    "max": 1,
    "palette": ["blue", "white", "green"],
}

# TODO: Add the NDVI and NDSI bands to the map
Map.addLayer(, ndvi_ndsi_vis_params, "NDVI 2021", False)
Map.addLayer(, ndvi_ndsi_vis_params, "NDSI 2021", False)
Map.addLayer(, ndvi_ndsi_vis_params, "NDVI 2022", False)
Map.addLayer(, ndvi_ndsi_vis_params, "NDSI 2022", False)

# TODO: Add the RGB composite images to the map
Map.addLayer(, vis_params, "Winter 2021 RGB", False)
Map.addLayer(, vis_params, "Winter 2022 RGB")

# Display the map
Map

Map(center=[28.350012666040698, 83.94499999999921], controls=(WidgetControl(options=['position', 'transparent_…

## Final Thoughts

In this project, we learned about two important indices used in remote sensing: NDVI and NDSI. We calculated the NDVI and NDSI values for a region of interest in the Himalayan region using Landsat 8 satellite imagery data and visualized the results.

NDVI and NDSI are valuable tools for monitoring vegetation health and snow cover, respectively. They provide insights into the health of ecosystems and can help in assessing the impact of climate change on food security and water resources.

In this example, we were able to use NDVI and NDSI to visualize the changes in vegetation and snow cover in the Himalayan region over the winter months of 2021 and 2022.